## Aim

Write a general ABC sampler, to understand the technique and to apply it to exoplanet occurrence problems.

In [1]:
import numpy as np
import scipy
import scipy.stats as stats
import functools
import sys
sys.path.append('..')
from dev.abc_generic import ABCSampler

In [2]:
# First simple example: a Gaussian fit you can do with MLE.
# Suppose we've sampled data from N(50, 10), but our best guess is it's N(N(40, 20), N(11, 2)).
prior = stats.multivariate_normal(np.array([40, 11]), np.diag([20, 2]))
def candidate_getter(p):
    def candidate(size):
        return np.random.normal(p[0], np.abs(p[1]), size)
    return candidate

gaussian_sampler = ABCSampler(prior, candidate_getter)
gaussian_sampler.sample(np.random.normal(50, 10, (100,)), verbose=False)

array([49.89086257,  9.74078142])

In [ ]:
# and now, PMC!
# let's do the same exact problem as in the first one
gaussian_sampler.sample_pmc(np.random.normal(50, 10, (100,)), [10, 5, 1, 0.5, 0.1, 0.01, 1e-3, 1e-4, 1e-5], 7)

Attempting to clear threshold 10
Attempting to clear threshold 5
Lowering threshold to 5.0
Lowering threshold to 5.0
Lowering threshold to 5.0
Lowering threshold to 5.0
Lowering threshold to 5.0
Lowering threshold to 5.0
Lowering threshold to 5.0
Attempting to clear threshold 1
Raising threshold to 2.23606797749979
Lowering threshold to 1.4953487812212205
Raising threshold to 1.8285790999795744
Lowering threshold to 1.3522496441040663
Lowering threshold to 1.1628626935730917
Lowering threshold to 1.0783611146425356
Raising threshold to 1.119815123320676
Raising threshold to 1.1411359387069382
Raising threshold to 1.151948094064055
Raising threshold to 1.1573925278485757
Raising threshold to 1.1601243866307458
Lowering threshold to 1.0770907049226384
Raising threshold to 1.117836836660008
Raising threshold to 1.1387843406384919
Lowering threshold to 1.06713838870059
Raising threshold to 1.1023794656770527
Lowering threshold to 1.049942601134487
Raising threshold to 1.07584160712914
Lowe

Raising threshold to 1.0433496817696397
Lowering threshold to 1.0214448990374565
Lowering threshold to 1.0106655723024587
Raising threshold to 1.016040940839058
Raising threshold to 1.0187393367458006
Lowering threshold to 1.0093261795603048
Lowering threshold to 1.0046522679814667
Lowering threshold to 1.002323434816061
Lowering threshold to 1.0011610433971454
Raising threshold to 1.001742070506105
Lowering threshold to 1.0008706562319154
Lowering threshold to 1.0004352334019007
Lowering threshold to 1.0002175930275874
Raising threshold to 1.0003264072957596
Lowering threshold to 1.0001631903323376
Lowering threshold to 1.0000815918375547
Lowering threshold to 1.0000407950866579
Raising threshold to 1.0000611932540722
Raising threshold to 1.0000713924938045
Raising threshold to 1.0000764921526772
Raising threshold to 1.0000790419918655
Raising threshold to 1.0000803169138974
Raising threshold to 1.000080954375523
Lowering threshold to 1.0000404763685933
Raising threshold to 1.00006071

Raising threshold to 0.5000334876614739
Lowering threshold to 0.5000167435503905
Raising threshold to 0.5000251155358445
Raising threshold to 0.5000293015811371
Lowering threshold to 0.5000146505759292
Raising threshold to 0.5000219760248725
Lowering threshold to 0.5000109878917025
Lowering threshold to 0.5000054939156682
Lowering threshold to 0.5000027469502883
Lowering threshold to 0.5000013734732578
Lowering threshold to 0.5000006867361573
Raising threshold to 0.7071072667821745
Raising threshold to 0.8408967039905523
Lowering threshold to 0.6484198886487645
Raising threshold to 0.7384132631302521
Raising threshold to 0.787990659303229
Raising threshold to 0.8140139729657148
Raising threshold to 0.8273461590344903
Lowering threshold to 0.6431742217449679
Lowering threshold to 0.5670865109244655
Lowering threshold to 0.5324877984162949
Raising threshold to 0.549514920374094
Raising threshold to 0.5582315817793547
Lowering threshold to 0.5283141024898704
Raising threshold to 0.5430668

Raising threshold to 0.5001011627345299
Lowering threshold to 0.500050578809049
Raising threshold to 0.5000758701322031
Raising threshold to 0.5000885162734658
Lowering threshold to 0.5000442561781236
Raising threshold to 0.5000663857361207
Raising threshold to 0.5000774508823721
Raising threshold to 0.5000829835473133
Raising threshold to 0.5000857499027381
Raising threshold to 0.5000871330861891
Lowering threshold to 0.5000435646452163
Raising threshold to 0.5000653483912124
Raising threshold to 0.5000762406200756
Lowering threshold to 0.5000381188569906
Lowering threshold to 0.5000190590652474
Raising threshold to 0.5000285888703052
Raising threshold to 0.5000333538409443
Raising threshold to 0.5000357363432915
Lowering threshold to 0.5000178678523856
Raising threshold to 0.5000268020180221
Raising threshold to 0.5000312691607025
Raising threshold to 0.5000335027470084
Lowering threshold to 0.5000167510929051
Raising threshold to 0.5000251268498058
Raising threshold to 0.50002931478

Raising threshold to 0.5000295637683169
Raising threshold to 0.5000305492573652
Raising threshold to 0.5000310420026177
Lowering threshold to 0.5000155207604149
Raising threshold to 0.5000232813212919
Lowering threshold to 0.5000116405251441
Raising threshold to 0.5000174608893422
Raising threshold to 0.500020371096848
Raising threshold to 0.5000218262069528
Raising threshold to 0.5000225537635931
Raising threshold to 0.5000229175423102
Raising threshold to 0.5000230994317679
Lowering threshold to 0.500011549582491
Lowering threshold to 0.5000057747578976
Lowering threshold to 0.5000028873706119
Attempting to clear threshold 0.1
Raising threshold to 0.22360679774997896
Raising threshold to 0.334370152488211
Lowering threshold to 0.18285790999795745
Raising threshold to 0.24726954371635135
Raising threshold to 0.2875405276445857
Raising threshold to 0.31007252389571827
Raising threshold to 0.32199223142401395
Raising threshold to 0.32812282992999264
Raising threshold to 0.33123176278635

...

Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.01480029002

Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.014800290025958998
Raising threshold to 0.01480029002

Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshol

Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshol

Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459
Raising threshold to 0.01479887343686459


In [ ]:
# Exp(22) but we estimate Exp(max(0, N(21, 2)))
prior = stats.halfnorm(21, 2)
def candidate_getter(p):
    def candidate(size):
        return np.random.exponential(p, size)
    return candidate

exp_sampler = ABCSampler(prior, candidate_getter)
exp_sampler.sample(np.random.exponential(22, (100,)), threshold=0.001, verbose=False)

1. https://arxiv.org/pdf/1802.09720.pdf